# Duns Normalization Pipeline
This notebook is to normalize (i.e. simplify) the duns file received from DnB prepare a file
- with each row corresponding to a duns record
- each record has identification (dunsid, name, others ids), localization (city, country..)
- each record has the direct parent (either found originally in DnB file (duns3), or calculated)

### Read the file

In [105]:
filepath='../1-AD/3-DUNS/'
filename='2017-10-19 DUNS Suppliers_2POR.xlsx'
df=pd.read_excel(filepath+filename)

df2=df.sample(1000)[['dunsnumber_01', 'name_01', 'name_02', 'street_01a', 'city_01',
       'countryname_01', 'countrycode_01', 'registerid_01', 'dunsnumber_03',
       'name_03', 'street_03', 'city_03', 'countryname_03', 'countrycode_03',
       'zip_03', 'dunsnumber_04', 'name_04', 'street_04', 'city_04',
       'countrycode_04', 'dunsnumber_05', 'name_05', 'street_05', 'city_05',
       'countryname_05', 'countrycode_05']]
df2.to_csv(filepath+'duns_sample.csv',encoding='utf-8',sep='|',index=False)
#df=pd.read_csv('duns_sample.csv',encoding='utf-8',sep='|')

In [106]:
df.isnull().sum()/df.shape[0]

dunsnumber_01                                                                      0.000000
name_01                                                                            0.000004
name_02                                                                            0.700492
street_01a                                                                         0.006555
city_01                                                                            0.000895
countryname_01                                                                     0.000004
countrycode_01                                                                     0.000013
registerid_01                                                                      0.230801
dunsnumber_03                                                                      0.000004
name_03                                                                            0.545998
street_03                                                                       

# Create a dictionnary with all available information on duns

### Concatenate the names of the different name columns (name 1,2)

In [107]:
def concatenate_names(myrow):
    '''
    This small function concatenate the different company names found across the names columns of SAP (name1, name2..)
    It takes the name found in the first column. If the name in the second column adds information to the first, 
    it concatenates (by adding it in brackets). And it continues like this for the other columns
    example:
    name1='KNIGHT FRANK (SA) PTY LTD'
    name2='KNIGHT FRANK'
    name3='ex-batman'
    name4='kapis code 3000'
    concatenate_names([name1,name2,name3,name4]):
        'KNIGHT FRANK (SA) PTY LTD (ex-batman, kapis code 3000)'
    '''
    from neatmartinet import compare_tokenized_strings
    from neatmartinet import normalizechars
    r = pd.Series(myrow,index=range(len(myrow)))
    r.dropna(inplace=True)
    if r.shape[0]==0:
        return None
    elif r.shape[0]==1:
        return r[0]
    else:
        s = r[0]
        for ix in range(1,len(r)):
            s1=normalizechars(s)
            r1=normalizechars(r[ix])
            score=compare_tokenized_strings(s1,r1)
            if pd.isnull(score) or score <0.8:
                if len(s)==len(r[0]):
                    s=s+' ('+r[ix]+')'
                else:
                    s=s.rstrip(')')+', '+r[ix]+')'
        return s
df['concatenatedname']=df[['name_01','name_02']].apply(lambda r:concatenate_names(r.values),axis=1)

In [108]:
df.columns

Index(['dunsnumber_01', 'name_01', 'name_02', 'street_01a', 'city_01',
       'countryname_01', 'countrycode_01', 'registerid_01', 'dunsnumber_03',
       'name_03', 'street_03', 'city_03', 'countryname_03', 'countrycode_03',
       'zip_03', 'dunsnumber_04', 'name_04', 'street_04', 'city_04',
       'countrycode_04', 'dunsnumber_05', 'name_05', 'street_05', 'city_05',
       'countryname_05', 'countrycode_05', 'out of bus',
       'Supplier Number - EADS Supplier Parent D-U-N-S® Number (enr)',
       'Supplier Number - EADS Supplier Parent Name (enr)',
       'Supplier Number - EADS Supplier Parent Country (enr)',
       'Supplier Number - EADS Supplier Domestic Ultimate Parent D-U-N-SÂ® Number (enr)',
       'Supplier Number - EADS Supplier Domestic Ultimate Parent Name (enr)',
       'Supplier Number - EADS Supplier Domestic Ultimate Parent City (enr)',
       'Supplier Number - EADS Supplier Domestic Ultimate Parent Country (enr)',
       'Supplier Number - EADS Supplier Global Ult

In [109]:
df=df.loc[df['dunsnumber_01']!='NDM999999']

### Re-normalize the duns information
this is without the parent ids

In [110]:
duns1_coldict={
    'dunsnumber_01':'dunsid',
    'concatenatedname':'companyname',
    'street_01a':'street',
    'city_01':'city',
    'zip_01':'postalcode',
    'countryname_01':'countryname',
    'countrycode_01':'countrycode',
    'registerid_01':'registerid'   
}
duns3_coldict={
    'dunsnumber_03':'dunsid',
    'name_03':'companyname',
    'street_03':'street',
    'city_03':'city',
    'zip_03':'postalcode',
    'countryname_03':'countryname',
    'countrycode_03':'countrycode',
    'registerid_03':'registerid'   
}
duns4_coldict={
    'dunsnumber_04':'dunsid',
    'name_04':'companyname',
    'street_04':'street',
    'city_04':'city',
    'zip_04':'postalcode',
    'countryname_04':'countryname',
    'countrycode_04':'countrycode',
    'registerid_04':'registerid'   
}
duns5_coldict={
    'dunsnumber_05':'dunsid',
    'name_05':'companyname',
    'street_05':'street',
    'city_05':'city',
    'zip_05':'postalcode',
    'countryname_05':'countryname',
    'countrycode_05':'countrycode',
    'registerid_05':'registerid'   
}

referencedata=pd.DataFrame(columns=list(duns1_coldict.values())+['source_level'])

#duns1
#select from possible columns the ones that are in the dataframe
temp=df[np.intersect1d(list(duns1_coldict.keys()),df.columns)]
print('missing cols 1:',[c for c in duns1_coldict.keys() if not c in df.columns])
temp.rename(columns=duns1_coldict,inplace=True)
temp['source_level']='duns1'
temp['dunsid']=temp['dunsid'].astype(int)
#drop the duplicates
temp=temp.drop_duplicates(subset=['dunsid'])
print(temp.shape[0])
referencedata=pd.concat([referencedata,temp]).copy()

#duns3
temp=df[np.intersect1d(list(duns3_coldict.keys()),df.columns)].copy()
print('missing cols 3:',[c for c in duns3_coldict.keys() if not c in df.columns])
temp.rename(columns=duns3_coldict,inplace=True)
temp['source_level']='duns3'
temp['dunsid']=temp['dunsid'].astype(int)
temp.drop_duplicates(subset=['dunsid'],inplace=True)
#only take the ones who are not already in the dataframe
temp=temp.loc[temp['dunsid'].isin(referencedata['dunsid'])==False]
print(temp.shape[0])
referencedata=pd.concat([referencedata,temp]).copy()

#duns4
temp=df[np.intersect1d(list(duns4_coldict.keys()),df.columns)].copy()
print('missing cols 4:',[c for c in duns4_coldict.keys() if not c in df.columns])
temp.rename(columns=duns4_coldict,inplace=True)
temp['source_level']='duns4'
temp['dunsid']=temp['dunsid'].astype(int)
temp.drop_duplicates(subset=['dunsid'],inplace=True)
#only take the ones who are not already in the dataframe
temp=temp.loc[temp['dunsid'].isin(referencedata['dunsid'])==False]
print(temp.shape[0])
referencedata=pd.concat([referencedata,temp]).copy()

#duns5
temp=df[np.intersect1d(list(duns5_coldict.keys()),df.columns)].copy()
print('missing cols 5:',[c for c in duns5_coldict.keys() if not c in df.columns])
temp.rename(columns=duns5_coldict,inplace=True)
temp['source_level']='duns5'
temp['dunsid']=temp['dunsid'].astype(int)
temp.drop_duplicates(subset=['dunsid'],inplace=True)
#only take the ones who are not already in the dataframe
temp=temp.loc[temp['dunsid'].isin(referencedata['dunsid'])==False]
print(temp.shape[0])


referencedata=pd.concat([referencedata,temp]).copy()
print(referencedata.shape[0])

missing cols 1: ['zip_01']


/Users/paulogier/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)
/Users/paulogier/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/paulogier/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ver

232507
missing cols 3: ['registerid_03']
5613
missing cols 4: ['zip_04', 'countryname_04', 'registerid_04']
428
missing cols 5: ['zip_05', 'registerid_05']
208
238756


In [111]:
#re-order columns
referencedata=referencedata[['dunsid','companyname','countrycode','countryname','street','city','postalcode','registerid','source_level']]

In [112]:
del temp
del duns1_coldict, duns3_coldict, duns4_coldict,duns5_coldict

In [113]:
countriesdict=pd.read_excel(filepath+'dunscountries.xlsx')
countriesdict=countriesdict.set_index('countryname')['countrycode']
countriesdict['NAMIBIA']='NA'

In [114]:
referencedata.loc[referencedata['countrycode'].isnull(),'countrycode']=referencedata.loc[
    referencedata['countrycode'].isnull(),'countryname'].replace(countriesdict)

In [115]:
countriesdict=countriesdict.reset_index(drop=False)
countriesdict=countriesdict.set_index('countrycode')['countryname']
referencedata.loc[referencedata['countryname'].isnull(),'countryname']=referencedata.loc[
    referencedata['countryname'].isnull(),'countrycode'].replace(countriesdict)

In [116]:
referencedata.isnull().sum()

dunsid               0
companyname          0
countrycode          0
countryname          0
street            1575
city               228
postalcode      233269
registerid       59911
source_level         0
dtype: int64

In [117]:
referencedata.set_index('dunsid',inplace=True)
referencedata.to_csv('allduns_information_nohierarchy',encoding='utf-8',sep='|')

# Re-create the parent information

### Select only the hierarchical columns

In [118]:
dunscols=['dunsnumber_01','dunsnumber_03','dunsnumber_04','dunsnumber_05']
fat=df[dunscols].copy()
for c in dunscols:
    fat[c]=fat[c].astype(int)

In [119]:
fat['countrycode_01']=fat['dunsnumber_01'].apply(lambda r:referencedata.loc[r,'countrycode'])
fat['countrycode_03']=fat['dunsnumber_03'].apply(lambda r:referencedata.loc[r,'countrycode'])
fat['countrycode_04']=fat['dunsnumber_04'].apply(lambda r:referencedata.loc[r,'countrycode'])
fat['countrycode_05']=fat['dunsnumber_05'].apply(lambda r:referencedata.loc[r,'countrycode'])

### Fill in the blanks

In [120]:
# Case duns3 is empty: fill with duns1
fat.loc[fat['dunsnumber_03'].isnull(),'dunsnumber_03']=fat.loc[fat['dunsnumber_03'].isnull(),'dunsnumber_01']

In [121]:
fat.isnull().sum()

dunsnumber_01     0
dunsnumber_03     0
dunsnumber_04     0
dunsnumber_05     0
countrycode_01    0
countrycode_03    0
countrycode_04    0
countrycode_05    0
dtype: int64

### Hierarchy: duns1
take 1 to 3 relation

In [122]:
parentdata=pd.DataFrame(columns=['dunsid','parentdunsid','source_relation'])
x=df[['dunsnumber_01','dunsnumber_03']].copy().rename(columns={
        'dunsnumber_01':'dunsid','dunsnumber_03':'parentdunsid'
    })
x['source_relation']='DnB original'
parentdata=pd.concat([parentdata,x]).copy()

### Hierarchy: duns5
take 5 to self relation

In [123]:
x=fat[['dunsnumber_05']].copy().rename(columns={
        'dunsnumber_05':'dunsid'
    })
x.drop_duplicates(subset=['dunsid'],inplace=True)
x=x.loc[x['dunsid'].isin(parentdata['dunsid'])==False]
x['parentdunsid']=x['dunsid']
x['source_relation']='DnB original'
parentdata=pd.concat([parentdata,x]).copy()

### Hiearchy: duns3
- if duns3 not already mapped in parentdata
- if duns3 and duns 4 are the same country
- then duns3 is ChildOf duns4

In [124]:
unknown3s=fat.loc[fat['dunsnumber_03'].isin(parentdata['dunsid'])==False,'dunsnumber_03'].unique()
x=fat.loc[(fat['dunsnumber_03'].isin(unknown3s)) & (fat['countrycode_04']==fat['countrycode_03']),
         ['dunsnumber_03','dunsnumber_04']].copy().rename(columns={
        'dunsnumber_03':'dunsid','dunsnumber_04':'parentdunsid'
    }).drop_duplicates()
x['source_relation']='DnB calc'
parentdata=pd.concat([parentdata,x]).copy()

- if duns3 not already mapped in parentdata
- if duns3 and duns4 are of different country
- if duns4 is duns1
- duns3 is child of duns5

In [125]:
unknown3s=fat.loc[fat['dunsnumber_03'].isin(parentdata['dunsid'])==False,'dunsnumber_03'].unique()
x=fat.loc[(fat['dunsnumber_03'].isin(unknown3s)) & (fat['dunsnumber_04']==fat['dunsnumber_01']),
         ['dunsnumber_03','dunsnumber_05']]
#y = x['dunsnumber_03'].value_counts()
#unique3s=y.loc[y==1].index
#x=x.loc[x['dunsnumber_03'].isin(unique3s)]
x=x.copy().rename(columns={
        'dunsnumber_03':'dunsid','dunsnumber_05':'parentdunsid'
    }).drop_duplicates()
x['source_relation']='DnB calc'
parentdata=pd.concat([parentdata,x]).copy()

- if duns3 not already mapped in parentdata
- if duns3 and duns4 are of different country
- duns3 is child of duns4

In [126]:
unknown3s=fat.loc[fat['dunsnumber_03'].isin(parentdata['dunsid'])==False,'dunsnumber_03'].unique()
x=fat.loc[(fat['dunsnumber_03'].isin(unknown3s)),
         ['dunsnumber_03','dunsnumber_04']]
#y = x['dunsnumber_03'].value_counts()
#unique3s=y.loc[y==1].index
#x=x.loc[x['dunsnumber_03'].isin(unique3s)]
x=x.copy().rename(columns={
        'dunsnumber_03':'dunsid','dunsnumber_04':'parentdunsid'
    }).drop_duplicates()
x['source_relation']='DnB calc'
parentdata=pd.concat([parentdata,x]).copy()

### Hierarchy: duns4
- the rest of the duns4 can be safely mapped to a duns5

In [127]:
unknown4s=fat.loc[fat['dunsnumber_04'].isin(parentdata['dunsid'])==False,'dunsnumber_04'].unique()
x=fat.loc[fat['dunsnumber_04'].isin(unknown4s),['dunsnumber_04','dunsnumber_05']]
x=x.copy().rename(columns={
        'dunsnumber_04':'dunsid','dunsnumber_05':'parentdunsid'
    }).drop_duplicates()
x['source_relation']='DnB calc'
parentdata=pd.concat([parentdata,x]).copy()

## Some Counting

In [128]:
unknown3s=fat.loc[fat['dunsnumber_03'].isin(parentdata['dunsid'])==False,'dunsnumber_03'].unique()
print(len(unknown3s))
unknown4s =fat.loc[(fat['dunsnumber_04'].isin(parentdata['dunsid'])==False) & (
    fat['dunsnumber_04']!=fat['dunsnumber_01']) ,
                  'dunsnumber_04'].unique()
print(len(unknown4s))
unknowns=list(set(list(unknown4s)+list(unknown3s)))
print(len(unknowns))

0
0
0


In [129]:
print(parentdata.shape)

(238756, 3)


In [142]:
parentdata['isglobalroot']=(parentdata['dunsid']==parentdata['parentdunsid'])
for c in ['dunsid','parentdunsid']:
    parentdata[c]=parentdata[c].astype(int)
parentdata.set_index('dunsid',inplace=True)

### Merge the two files

In [143]:
finaldata=referencedata.join(parentdata)
finaldata.to_csv('duns_denormalized.csv',index=True,sep='|',encoding='utf-8')
print(finaldata.shape)

(238756, 11)


In [146]:
finaldata.isnull().sum()

companyname             0
countrycode             0
countryname             0
street               1575
city                  228
postalcode         233269
registerid          59911
source_level            0
parentdunsid            0
source_relation         0
isglobalroot            0
dtype: int64